<h1 style='font-size:40px'> Machine Learning with MLlib</h1>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Iremos aprender a utilizar a spark.ml com um dataset sobre aluguéis do Airbnb em São Francisco.  
        </li>
    </ul>
</div>

<h2 style='font-size:30px'> Data Ingestion and Exploration</h2>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Tive que remover casas com preço 0.00 dólares e preencher valores nulos com a mediana das colunas. &#x24;'s foram deletados das colunas de preço também.
        </li>
    </ul>
</div>

In [1]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/databricks/LearningSparkV2/master/databricks-datasets/learning-spark-v2/sf-airbnb/sf-airbnb.csv',
                usecols=['neighbourhood', 'room_type', 'bedrooms', 'bathrooms', 'number_of_reviews', 'price'])

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('C.10').getOrCreate()

22/06/15 13:14:12 WARN Utils: Your hostname, veiga-Inspiron resolves to a loopback address: 127.0.1.1; using 192.168.15.21 instead (on interface wlp7s0)
22/06/15 13:14:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/15 13:14:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
airbnbDF = spark.createDataFrame(df, schema='''`neighborhood` STRING, `room_type` STRING, `bathrooms` FLOAT, `bedrooms` FLOAT, `price` STRING, 
                            `number_of_reviews` INT''')

In [10]:
from pyspark.ml.feature import Imputer
import pyspark.sql.functions as F

num_cols = ['bathrooms', 'bedrooms', 'price', 'number_of_reviews']
imputer = Imputer(strategy='median', inputCols=num_cols, outputCols=[f'{col}_na' for col in num_cols])
airbnbDF = (airbnbDF.withColumn('price', F.substring('price', 2, 10).cast('float'))
        .where('price>0'))

airbnbDF = imputer.fit(airbnbDF).transform(airbnbDF)
airbnbDF = airbnbDF.drop('bathrooms', 'bedrooms', 'price', 'number_of_reviews')
airbnbDF.show()

+--------------------+---------------+------------+-----------+--------+--------------------+
|        neighborhood|      room_type|bathrooms_na|bedrooms_na|price_na|number_of_reviews_na|
+--------------------+---------------+------------+-----------+--------+--------------------+
|     Duboce Triangle|Entire home/apt|         1.0|        1.0|   170.0|                 180|
|      Bernal Heights|Entire home/apt|         1.0|        2.0|   235.0|                 111|
|         Cole Valley|   Private room|         4.0|        1.0|    65.0|                  17|
|         Cole Valley|   Private room|         4.0|        1.0|    65.0|                   8|
|Western Addition/...|Entire home/apt|         1.5|        2.0|   785.0|                  27|
|Western Addition/...|Entire home/apt|         1.0|        2.0|   255.0|                  31|
|    Mission District|   Private room|         1.0|        1.0|   139.0|                 647|
|        Potrero Hill|   Private room|         1.0|        1

<h2 style='font-size:30px'> Creating Training and Test Sets</h2>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            O autor ensina o funcionamento de um train-test-split e o implanta em "AirbnbDF".
        </li>
    </ul>
</div>

In [13]:
# Criação dos sets de treino e teste.
trainDF, testDF = airbnbDF.randomSplit([0.8, 0.2], seed=42)

<h2 style='font-size:30px'> Preparing Features with Transformers</h2>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            O aprendizado dos algoritmos no Spark é um tanto diferente quando comparado aos do scikit-learn. Por aqui, os modelos exigem que todas as features de uma estejam contidas dentro de um vetor. Nesse sentido, é necessário criar nos datasets uma coluna adicional na qual serão adicionados os vetores.
        </li>
    </ul>
</div>

In [ ]:
# Essa tarefa contará com o transformador 'VectorAssembler'.

<p style='color:red'> Creating Training and Test Data Sets